In [1]:
from datetime import date
date.today()

datetime.date(2017, 12, 10)

In [2]:
author = "NirantK. https://github.com/NirantK/keras-practice"
print(author)

NirantK. https://github.com/NirantK/keras-practice


In [3]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
import keras

Using TensorFlow backend.


In [5]:
keras.__version__

'2.0.8'

In [6]:
import os
if os.name=='nt':
    print('We are on Windows')

We are on Windows


Prerequisites: 
- Python programming experience
- Familiarity with machine learning vocabulary
   * Refer [Ailyen Blog](http://blog.aylien.com/10-machine-learning-terms-explained-in-simple/) - 5 minutes read or [Google's Machine Learning Glossary](https://developers.google.com/machine-learning/glossary/) when you see/hear new terms
    
Nice to have (will build upon these topics)
- Familiarity with deep feed forward networks
- Understand backpropagation 
- Familiarity with activation functions like relu, tanh, softmax, hierarchical softmax

**We will start with an introduction to the theory behind convnets, specifically:**

- What is convolution and max-pooling?   
- What are convnets?
- What do convnets learn?

Then we will cover image classification with **small datasets**:

- Training your own small convnets from scratch
- Using data augmentation to mitigate overfitting
- Using a pre-trained convnet to do feature extraction
- Fine-tuning a pre-trained convnet

We will use our convnet to classify **MNIST digits**

## In this notebook: 
- MNIST Demo
    - Using Convolution Layers
- Code Overview

### In next notebook:
- What is convolution and maxpooling? 
- What are convnets? 
- What do convnets learn? 

In [9]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

The 6 lines of code show you what a basic convnet looks like. It’s a stack of ```Conv2D``` and ```MaxPooling2D``` layers. 

**We’ll see in a minute what they do concretely**

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


The next step would be to feed our last output tensor (of shape (3, 3, 64)) into a densely-connected classifier network like those you are already familiar with: *a stack of Dense layers*. 

These classifiers process vectors, which are 1D, whereas our current output is a 3D tensor. So first, we will have to flatten our 3D outputs to 1D, and then add a few Dense layers on top:

In [11]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                36928     
__________

In [14]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 7s - loss: 0.1828 - acc: 0.9427     
Epoch 2/5
60000/60000 [==============================] - 5s - loss: 0.0493 - acc: 0.9846     
Epoch 3/5
60000/60000 [==============================] - 5s - loss: 0.0330 - acc: 0.9892     
Epoch 4/5
60000/60000 [==============================] - 5s - loss: 0.0256 - acc: 0.9919     
Epoch 5/5
60000/60000 [==============================] - 6s - loss: 0.0196 - acc: 0.9942     


# Model Performance
---
Show off step

In [15]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 0s     


0.99219999999999997